# 📓 02_team_simulation.ipynb
**Purpose:** Simulate Day 2 team scores based on player sampling and calculate win probabilities.

In [ ]:
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team

In [ ]:
# --- Load Data ---

def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

PICKLE_FILE = 'golf_data.pkl'  
players, tournaments = load_data(PICKLE_FILE)
print(f"✅ Loaded {len(players)} players and {len(tournaments)} tournaments.")

In [ ]:
# --- Sampling Functions ---

def get_net_scores(player):
    return [r.net for r in player.rounds]

def sample_score_simple(player):
    net_scores = get_net_scores(player)
    return random.choice(net_scores) if net_scores else None

def sample_score_weighted(player, decay_factor=0.9):
    net_scores = get_net_scores(player)
    n = len(net_scores)
    if n == 0:
        return None
    weights = [decay_factor ** (n - i - 1) for i in range(n)]
    return random.choices(net_scores, weights=weights, k=1)[0]

In [ ]:
# --- Simulate Single Team Score ---

def simulate_team_score(team, sampling_fn=sample_score_simple):
    """
    Given a Team object, simulate one Day 2 total net score.
    Sampling function can be simple or weighted.
    """
    player_scores = []
    for player in team.members:
        sampled_score = sampling_fn(player)
        if sampled_score is not None:
            player_scores.append(sampled_score)

    if len(player_scores) < 4:
        # Not enough valid scores, team cannot score (return high number)
        return 999

    # Best 4 of 6 scores count
    best_4_scores = sorted(player_scores)[:4]
    return sum(best_4_scores)

In [ ]:
# --- Simulate Entire Tournament ---

def simulate_tournament(teams, num_simulations=10000, sampling_fn=sample_score_simple):
    """
    Simulate many tournaments and estimate win percentages for each team.
    """
    team_wins = {team.name: 0 for team in teams}

    for _ in range(num_simulations):
        team_totals = {}
        for team in teams:
            player_scores = []
            for player in team.members:
                sampled_score = sampling_fn(player)
                if sampled_score is not None:
                    player_scores.append(sampled_score)
            if len(player_scores) >= 4:
                best_4_scores = sorted(player_scores)[:4]
                team_total = sum(best_4_scores)
            else:
                team_total = float('inf')  # Not enough players

            team_totals[team.name] = team_total

        # Find team with the lowest total
        winning_team = min(team_totals, key=team_totals.get)
        team_wins[winning_team] += 1

    # Convert wins to win percentages
    team_win_percentages = {team: (wins / num_simulations) * 100 for team, wins in team_wins.items()}

    return team_win_percentages

In [ ]:
# --- Simulate Example Tournament ---
team_win_percentages = simulate_tournament(example_teams, num_simulations=10000, sampling_fn=sample_score_simple)

# --- Print Win Probabilities ---
print("\n🏆 Team Win Percentages (Estimated):")
for team_name, win_pct in team_win_percentages.items():
    print(f"{team_name:<15}: {win_pct:.2f}%")

In [ ]:
# --- Example Setup ---

# TODO: Load your actual teams here.
# Example dummy setup for now

class Team:
    def __init__(self, name, members):
        self.name = name
        self.members = members  # list of Player objects

def print_team_simulation(team, sampling_fn=sample_score_simple):
    """
    Print team member scores, sorted best to worst,
    and show the top 4 scores that count toward team total.
    """
    sampled_scores = []
    
    for player in team.members:
        sampled_score = sampling_fn(player)
        if sampled_score is not None:
            sampled_scores.append((player.name, sampled_score))
        else:
            sampled_scores.append((player.name, float('inf')))  # Treat missing scores as very bad
    
    # Sort players by score (low to high)
    sampled_scores.sort(key=lambda x: x[1])

    print(f"\nTeam: {team.name}")
    print(f"{'Player':<25} {'Sampled Score':>15}")
    print("-" * 40)
    for name, score in sampled_scores:
        print(f"{name:<25} {score:>15}")
    
    # Top 4 scores
    top4 = sampled_scores[:4]
    top4_total = sum(score for _, score in top4)
    
    print("\nTop 4 Scores that Count:")
    for name, score in top4:
        print(f"{name:<25} {score:>15}")
    print(f"\nTeam Total (Best 4 Scores): {top4_total}\n")

# --- Create Example Teams (Random Selection) ---
example_teams = [
    Team("Team A", random.sample(list(players.values()), 6)),
    Team("Team B", random.sample(list(players.values()), 6))
]

# --- Test Print Example Teams ---
for team in example_teams:
    print_team_simulation(team, sampling_fn=sample_score_simple)

In [ ]:
def create_random_teams(players, num_teams=20, players_per_team=6):
    """
    Create random teams from the player pool.
    """
    player_list = list(players.values()) if isinstance(players, dict) else players
    random.shuffle(player_list)
    
    teams = []
    for i in range(num_teams):
        team_members = random.sample(player_list, players_per_team)
        team_name = f"Team {i+1}"
        teams.append(Team(name=team_name, members=team_members))
    
    return teams

In [ ]:
def simulate_tournament_with_scores(teams, num_simulations=10000, sampling_fn=sample_score_simple):
    """
    Simulate tournaments and collect average team scores.
    """
    team_total_scores = {team.name: [] for team in teams}

    for _ in range(num_simulations):
        for team in teams:
            player_scores = []
            for player in team.members:
                sampled_score = sampling_fn(player)
                if sampled_score is not None:
                    player_scores.append(sampled_score)
            if len(player_scores) >= 4:
                best_4_scores = sorted(player_scores)[:4]
                team_total = sum(best_4_scores)
            else:
                team_total = float('inf')  # Not enough players

            team_total_scores[team.name].append(team_total)

    # Calculate average team scores
    team_avg_scores = {team_name: np.mean(scores) for team_name, scores in team_total_scores.items()}
    
    return team_avg_scores

In [ ]:
# --- Create Random Teams ---
example_random_teams = create_random_teams(players, num_teams=20, players_per_team=6)

# --- Simulate Tournament ---
team_avg_scores = simulate_tournament_with_scores(example_random_teams, num_simulations=10000, sampling_fn=sample_score_simple)


print("\n🏆 Top 8 Teams by Expected Score:")

top_8 = sorted(team_avg_scores.items(), key=lambda x: x[1])[:8]

for rank, (team_name, avg_score) in enumerate(top_8, start=1):
    # Find the team object
    team_obj = next(team for team in example_random_teams if team.name == team_name)
    
    member_names = [player.name for player in team_obj.members]
    member_names_str = ', '.join(member_names)
    
    print(f"{rank}. {team_name:<10} - Expected Team Score: {avg_score:.2f}")
    print(f"    Members: {member_names_str}\n")